In [1]:
# OMR Model 
# Goal: recognize images of music excerpts

# Modules
import torch
from torch.autograd import Variable
import numpy as np
import pylab as pl
import torch.nn.init as init
import torch.optim as optim
import torch.nn as nn
import cv2
import ctc_utils
from primus import CTC_PriMuS

import matplotlib as mpl

class cnn_model(torch.nn.Module):
    def __init__(self, batch_size):
        super(cnn_model, self).__init__()

        kernel_size = [3,3]

        self.conv1 = nn.Conv2d(1, 16, kernel_size = kernel_size)
        self.batch1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16,32, kernel_size = kernel_size)
        self.batch2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32,64, kernel_size = kernel_size)
        self.batch3 = nn.BatchNorm2d(64)

        self.act = nn.LeakyReLU()
        self.pool = nn.MaxPool2d(2,2)

    def forward(self, x):

        # FORWARD PASS
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.act(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.batch2(x)
        x = self.act(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.batch3(x)
        x = self.act(x)
        x = self.pool(x)

        output = x

        return x

In [2]:
class ImageRNN(nn.Module):
    def __init__(self, batch_size = 16, n_inputs = 896, n_neurons = 4, n_outputs = vocabulary_size +1): # N_ STEPS AFTER BATCH_SIZE
        super(ImageRNN, self).__init__()
        
        self.n_neurons = n_neurons
        self.batch_size = batch_size
        #self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons) 
        
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
        self.log_softmax = nn.LogSoftmax(dim=1)
        
    def init_hidden(self, batch_size):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, batch_size, self.n_neurons))
        
    def forward(self, X):
        # transforms X to dimensions: n_steps X batch_size X n_inputs
        #X = X.permute(1, 0, 2) 
        
        self.batch_size = X.size(1)
        self.hidden = self.init_hidden(self.batch_size)
        
        lstm_out, self.hidden = self.basic_rnn(X, self.hidden)      
        out = self.FC(self.hidden)
        
        # Add softmax layer to get log probs
        out = self.log_softmax(out)
        
            
        return out#.view(-1, self.n_outputs) # batch_size X n_output
    
    
    #output represent log prob of model

NameError: name 'vocabulary_size' is not defined

In [4]:
# Data
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
corpus = './Data/package'# PATH
set = 'Data/train.txt' 
vocabulary = 'Data/vocabulary_semantic.txt'  
save_model = './trained_\semantic_model'

primus = CTC_PriMuS(corpus, set, vocabulary, semantic = True, val_split = 0.1)
primus.training_list

Training with 70880 and validating with 7875


['000121173-1_1_1',
 '210237506-1_1_1',
 '000104453-1_2_1',
 '000111912-1_1_1',
 '000110669-1_1_1',
 '000107077-1_1_1',
 '000104786-1_1_1',
 '211005066-1_1_1',
 '211005517-1_4_1',
 '212003804-1_1_1',
 '230001806-1_1_1',
 '212003609-1_3_1',
 '220019620-1_10_1',
 '220016675-1_1_2',
 '000122122-1_1_1',
 '210022171-1_2_1',
 '000117336-1_1_1',
 '000113033-1_1_1',
 '212001214-1_3_1',
 '000114742-1_1_1',
 '212001098-1_1_1',
 '190005785-1_1_1',
 '212002049-1_2_1',
 '000113968-1_1_1',
 '000120498-1_1_2',
 '210000268-1_2_1',
 '000121449-1_1_1',
 '220016913-1_25_2',
 '000125813-1_1_2',
 '000100134-9_1_1',
 '211007098-1_10_2',
 '230006366-1_3_1',
 '000135215-1_1_1',
 '000106117-3_1_1',
 '000114964-1_1_2',
 '225003319-1_1_1',
 '000109944-1_1_1',
 '000109820-1_1_1',
 '000118856-1_2_2',
 '190005578-1_1_1',
 '000109717-1_1_1',
 '150203302-1_1_1',
 '220017018-1_8_2',
 '000114847-1_1_2',
 '000112653-1_1_1',
 '100500581-1_1_1',
 '000117629-1_1_2',
 '110000324-1_1_1',
 '000106559-1_1_1',
 '230002314-1_1_1

In [5]:
# Variables
img_height = 128
N_EPOCHS = 10
dropout = 0.5

BATCH_SIZE = 16
vocabulary_size = primus.vocabulary_size

In [6]:
# Loop RNN
# num steps: IMAGE WIDTH
# batch size 16
# n_inputs 64 * 14 (from CNN output)
# output of CNN: (64 by 14 by width) - width same across batch

class LoopRNN(nn.Module):
    def __init__(self, batch_size = 16, n_inputs = 896, n_neurons = 4, n_outputs = vocabulary_size +1): # N_ STEPS AFTER BATCH_SIZE
        super(LoopRNN, self).__init__()
        
        self.n_neurons = n_neurons
        #self.batch_size = batch_size
        #self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons) 
        
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
    def init_hidden(self, batch_size):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, batch_size, self.n_neurons))
        
    def forward(self, X):
        # transforms X to dimensions: n_steps X batch_size X n_inputs
        #X = X.permute(1, 0, 2) 
        # maybe batch size should be width
        # each batch is 1 by 64 by 14
        
        self.batch_size = X.size(2)
        self.hidden = self.init_hidden(self.batch_size)
        
        # try using a loop - delete this if it breaks
        #lstm_out, self.hidden = self.basic_rnn(X, self.hidden)      
        #out = self.FC(self.hidden)
        out = []
        
        for x in X:
            lstm_out, self.hidden = self.basic_rnn(x, self.hidden)
            out_step = self.FC(self.hidden)
            out.append(out_step)

            
        return out#.view(-1, self.n_outputs) # batch_size X n_output

In [7]:
# Default params
# With image height of 128, width will be 1870
params = dict()
params['img_height'] = img_height
params['img_width'] = None
params['batch_size'] = 16
params['img_channels'] = 1
params['conv_blocks'] = 4
params['conv_filter_n'] = [32, 64, 128, 256]
params['conv_filter_size'] = [ [3,3], [3,3], [3,3], [3,3] ]
params['conv_pooling_size'] = [ [2,2], [2,2], [2,2], [2,2] ]
params['rnn_units'] = 512
params['rnn_layers'] = 2
params['vocabulary_size'] = vocabulary_size
params['max_width'] = 1500


In [8]:
# SETUP

learning_rate = 0.000001
criterion = torch.nn.CTCLoss()
model_cnn = cnn_model(BATCH_SIZE)
model_rnn = ImageRNN()
optimizer = optim.Adam(list(model_cnn.parameters()) + list(model_rnn.parameters()), lr = learning_rate)
len_data = len(primus.training_list) + len(primus.validation_list)


NameError: name 'ImageRNN' is not defined

In [9]:
# check batch targets and inputs for format
# batch targets is a list of target sequences
batch = primus.nextBatch(params)
batch['targets'][0]

# targets from ctc_utils.sparse_tuple_from
# is a tensor with an array of targets all put together
targets = ctc_utils.sparse_tuple_from(batch['targets'])
targets[0]

# targets[0] is an array of lists that has [batch_index, sequence_index] for each target sequence
# targets[1] has the lists of all the target sequences
# targets[2] is a list holding the batch size and the max target sequence length -- ie [16,37]

array([[ 0,  0],
       [ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0,  6],
       [ 0,  7],
       [ 0,  8],
       [ 0,  9],
       [ 0, 10],
       [ 0, 11],
       [ 0, 12],
       [ 0, 13],
       [ 0, 14],
       [ 0, 15],
       [ 0, 16],
       [ 0, 17],
       [ 0, 18],
       [ 0, 19],
       [ 0, 20],
       [ 0, 21],
       [ 0, 22],
       [ 0, 23],
       [ 0, 24],
       [ 0, 25],
       [ 0, 26],
       [ 1,  0],
       [ 1,  1],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 1,  5],
       [ 1,  6],
       [ 1,  7],
       [ 1,  8],
       [ 1,  9],
       [ 1, 10],
       [ 1, 11],
       [ 1, 12],
       [ 1, 13],
       [ 1, 14],
       [ 1, 15],
       [ 1, 16],
       [ 1, 17],
       [ 1, 18],
       [ 1, 19],
       [ 1, 20],
       [ 1, 21],
       [ 1, 22],
       [ 1, 23],
       [ 1, 24],
       [ 1, 25],
       [ 1, 26],
       [ 1, 27],
       [ 1, 28],
       [ 1, 29],
       [ 1, 30],
       [ 1, 31

In [10]:
# CTC Loss (homemade from Medium article recipe)

def ctcloss_reference(log_probs, targets, input_lengths, target_lengths, blank=0, reduction='mean'):
    
    input_lengths = torch.as_tensor(input_lengths, dtype=torch.long)
    target_lengths = torch.as_tensor(target_lengths, dtype=torch.long)
    dt = log_probs.dtype
    log_probs = log_probs.double()
    targets = targets.long()
    cum_target_lengths = target_lengths.cumsum(0)
    losses = []
    
    for i in range(log_probs.size(1)):
        input_length = input_lengths[i].item()
        target_length = target_lengths[i].item()
        cum_target_length = cum_target_lengths[i].item()
        
        # fill target sequences with blank symbol
        targets_prime = targets.new_full((2 * target_length + 1,), blank)
        
        
        # Then fill every odd value with target symbol
        if targets.dim() ==2:
                targets_prime[1::2] = targets[i, :target_length]
                
        else:
            targets_prime[1::2] = targets[cum_target_length - target_length:cum_target_length]
        
        # convert original inputs from log space by exponentiating
        probs = log_probs[:input_length, i].exp()
        
        # the length is the same as the target sequences
        alpha = log_probs.new_zeros((target_length * 2 + 1,))
        alpha[0] = probs[0, blank]
        alpha[1] = probs[0, targets_prime[1]]
        
        # this mask is only true when a[current] != a[current - 2]
        # please note that every odd element is blank, so this condition never holds for them
        mask_third = (targets_prime[:-2] != targets_prime[2:])
        
        for t in range(1, input_length):
            alpha_next = alpha.clone()
            
            # we always add a[current-1] to a[current]
            alpha_next[1:] += alpha[:-1]
            
            # but we add a[current-2] to a[current] only when mask condition is true
            alpha_next[2:] += torch.where(mask_third, alpha[:-2], alpha.new_zeros(1))
            alpha = probs[t, targets_prime] * alpha_next
            
        # to evaluate maximum likelihodd error, we need the natural logs of the target labelling probs
        losses.append(-alpha[-2:].sum().log()[None])
        
        output = torch.cat(losses, 0)
        
        if reduction == 'mean':
            return (output / target_lengths.to(dtype=output.dtype, device=output.device)).mean()
        
        elif reduction == 'sum':
            return output.sum()
        
        output = output.to(dt)
        
        return output


In [59]:
# Train based on Medium article
# Check output sizes
# Train using ImageRNN

for epoch in range(N_EPOCHS):
    train_loss = 0.
    train_acc = 0.
    model_cnn.train()
    model_rnn.train()
    
    for i in range(0, len_data, BATCH_SIZE):
        # zero parameter gradients
        optimizer.zero_grad()
        
        
        # Get inputs
        batch = primus.nextBatch(params)

        data = batch['inputs'] # size (batch, height, width, channels)
        #print(data)
        #print(data.shape)
        max_input_length = data.shape[2]
        
        # list of indices, values, shape
        seq_len = int(batch['seq_lengths'][0])
        targets = ctc_utils.sparse_tuple_from(batch['targets'])
        #print(tuple(targets[2]))
        #print(type(t[0]) for t in targets)
        #targets = torch.sparse_coo_tensor(targets[0], targets[1], tuple(targets[2]))
        targets_0 = torch.as_tensor((targets[0]))
        #print(targets.shape)
        #targets = torch.reshape(targets, (16, 1))
        padded_targets, lengths = ctc_utils.pad_sequences(batch['targets'], maxlen=max_input_length)
        padded_targets_tensor = torch.tensor(padded_targets)
        
        tensor_data = torch.from_numpy(data)
        #print(tensor_data.shape)
        tensor_data_reshape = torch.permute(tensor_data,(0,3, 1, 2))
        
        # forward, backward, optim
        cnn_output = model_cnn(tensor_data_reshape)
        #output_size = 64 * 14 * cnn_output.shape[3]
        #print(cnn_output.shape)
        #print(cnn_output[0])
        print(cnn_output[0].shape)
        print(cnn_output[0][0].shape)
        
        # Change shape for rnn
        output = torch.reshape(cnn_output, (cnn_output.shape[3], 16, 64 * 14)) # width, batch, features
        
        # Try width reduction as in the paper
        #output = torch.reshape(cnn_output, (int(cnn_output.shape[3]/16), 16, int(64 * 14/16))) # width, batch, features
    
        print(output.shape)
        
        
        # reset hidden states
        model_rnn.hidden = model_rnn.init_hidden(BATCH_SIZE)
        logits = model_rnn(output)
        print(logits.shape)
        print(logits[0].shape) #1 by width by vocab size
        #print((logits[0][0].shape))
        #print(logits[0][0][0].shape) # this is logits for one column of picture
        #print(batch['seq_lengths'])
        '''        
        for t in range(0,len(logits)):
            logits[t] = torch.reshape(logits[t],(logits[t].size(1), logits[t].size(2)))
        print(logits[0].shape[0])
        
        batch_size = logits[0].shape[0]
        logits_tensor = torch.stack(logits)
        logits_tensor = torch.permute(logits_tensor, (1,0,2))
        '''
        #rnn_output_reshape = torch.reshape(rnn_output, (cnn_output[0].shape[2], BATCH_SIZE, N_OUTPUTS))
        #rnn_output_reshape = torch.reshape(rnn_output[0], (1, BATCH_SIZE, N_OUTPUTS))
        #rnn_output_reshape = rnn_output[0].view(-1, BATCH_SIZE, N_OUTPUTS)
        
        # change targets to 1D tensor
        
        list_of_target_tensors = []
        lens = 0
        for i in range(0,16):
            list_of_target_tensors.append(torch.as_tensor(batch['targets'][i]))
            lens += len(batch['targets'][i])
        list_of_target_tensors
        tensor_of_target_tensors = torch.cat(list_of_target_tensors)
        #tensor_of_target_tensors.size() 


        #log_probs = nn.functional.log_softmax(rnn_output, dim=0)
        #Input and target shape
        #print(rnn_output_reshape.shape)
        input_shape = (BATCH_SIZE, params['img_height'], tensor_data_reshape.shape[3],1)
        input_len = tuple([batch_size for i in range (0, 16)])
        #print(input_shape)
        print(input_len)
        #print(logits_tensor.shape)
        target_shape = tuple(int(b) for b in batch['seq_lengths'])
        
        # MUST BE TENSOR, TENSOR, TUPLE, TUPLE OR TENSOR TENSOR TENSOR TENSOR
        #loss = criterion(rnn_output_reshape, padded_targets_tensor, input_len, target_shape)
        #loss = criterion(log_probs, padded_targets_tensor, input_len, target_shape)
        #loss = criterion(logits_tensor, tensor_of_target_tensors, input_len, tuple(lengths))
        
        loss = criterion(logits, tensor_of_target_tensors, input_len, tuple(lengths))
        loss.backward()
        optimizer.step()
        
        train_loss += loss.detach().item()
        train_acc += 0
        print("Loss: %f", train_loss)
        
    #model.eval()
    print('training loss:')
    print(train_loss)

    

torch.Size([64, 14, 204])
torch.Size([14, 204])
torch.Size([204, 16, 896])
torch.Size([1, 16, 1782])
torch.Size([16, 1782])
(16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16)


RuntimeError: Expected input_lengths to have value at most 1, but got value 16 (while checking arguments for ctc_loss_cpu)

In [104]:
# Debug loss

# Get data and reshape for CNN
data = batch['inputs'] # size (batch, height, width, channels)

tensor_data = torch.from_numpy(data)

tensor_data_reshape = torch.permute(tensor_data,(0,3, 1, 2))

# Send data through CNN
cnn_output = model_cnn(tensor_data_reshape)

# Change shape for RNN
output = torch.reshape(cnn_output, (cnn_output.shape[3], 16, 64 * 14)) # width, batch, features

output_to_loop = torch.reshape(cnn_output, (16, 1, cnn_output.shape[3], 64*14))

# Send data through RNN
loop_model = LoopRNN()
model_rnn.hidden = model_rnn.init_hidden(BATCH_SIZE)
logits = model_rnn(output)

# Loop returns list of tensor of tensor of tensors ... ?
# loop_output[0][0][0] is the output probs for first input of first input in batch
loop_output = loop_model(output_to_loop)

print(logits.shape)
print(loop_output[0][0][0])

# Convert loop output to tensor of shape (batch_size, width, vocab+1)

for t in range(0,len(loop_output)):
        loop_output[t] = torch.reshape(loop_output[t],(loop_output[t].size(1), loop_output[t].size(2)))
        print(loop_output[0].shape[0])
        
batch_idx_size = loop_output[0].shape[0]
loop_tensor = torch.stack(loop_output)

# Reshape to correct shape for Medium article
loop_tensor = torch.permute(loop_tensor, (1,0,2))

print(loop_tensor.shape)

# Convert to log softmax

log_softmax = nn.LogSoftmax(dim=1)

loop_logits = log_softmax(loop_tensor)


# Final shape needs to be (batch size, width, vocab size + 1)
print(loop_logits.shape)

torch.Size([1, 16, 1782])
tensor([-0.9073, -0.8068,  0.1632,  ..., -0.1925,  0.1541, -0.4404],
       grad_fn=<SelectBackward0>)
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
torch.Size([204, 16, 1782])
torch.Size([204, 16, 1782])


In [105]:
# Check target and target lengths sizes
target_lengths = tuple(lengths)
input_lengths = tuple(int(b) for b in batch['seq_lengths'])
targets = tensor_of_target_tensors
log_probs = loop_logits


In [110]:
result = criterion(log_probs, targets, input_lengths, target_lengths)
expected = ctcloss_reference(log_probs, targets, input_lengths, target_lengths).float()

print("Result from Torch: %f" %result)
print("Custom Function: %f" %expected)

Result from Torch: 9.749756
Custom Function: 9.577264


In [11]:
learning_rate = 0.0001
criterion = torch.nn.CTCLoss()
model_cnn = cnn_model(BATCH_SIZE)
model_rnn = LoopRNN()
optimizer = optim.Adam(list(model_cnn.parameters()) + list(model_rnn.parameters()), lr = learning_rate)
len_data = len(primus.training_list)
log_softmax = nn.LogSoftmax(dim=1)

In [116]:

for epoch in range(N_EPOCHS):
    train_loss = 0.
    train_acc = 0.
    model_cnn.train()
    model_rnn.train()
    
    for i in range(0, len_data, BATCH_SIZE):
        # zero parameter gradients
        optimizer.zero_grad()
        
        # Get inputs
        batch = primus.nextBatch(params)

        data = batch['inputs'] # size (batch, height, width, channels)
        max_input_length = data.shape[2]
        padded_targets, lengths = ctc_utils.pad_sequences(batch['targets'], maxlen=max_input_length)

        tensor_data = torch.from_numpy(data)

        tensor_data_reshape = torch.permute(tensor_data,(0,3, 1, 2))
        
        # forward, backward, optim
        cnn_output = model_cnn(tensor_data_reshape)

        output_to_loop = torch.reshape(cnn_output, (16, 1, cnn_output.shape[3], 64*14))
        
        model_rnn.hidden = model_rnn.init_hidden(BATCH_SIZE)
        loop_output = model_rnn(output_to_loop)
        
        for t in range(0,len(loop_output)):
            loop_output[t] = torch.reshape(loop_output[t],(loop_output[t].size(1), loop_output[t].size(2)))

        loop_tensor = torch.stack(loop_output)

        # Reshape to correct shape for Medium article
        loop_tensor = torch.permute(loop_tensor, (1,0,2))


        # Convert to log softmax
        loop_logits = log_softmax(loop_tensor)
  
        #Target to tensor
        list_of_target_tensors = []
        lens = 0
        for i in range(0,16):
            list_of_target_tensors.append(torch.as_tensor(batch['targets'][i]))
            lens += len(batch['targets'][i])
        tensor_of_target_tensors = torch.cat(list_of_target_tensors)

        # Set parameters for loss
        target_lengths = tuple(lengths)
        input_lengths = tuple(int(b) for b in batch['seq_lengths'])
        targets = tensor_of_target_tensors
        log_probs = loop_logits

        result = criterion(log_probs, targets, input_lengths, target_lengths)
        expected = ctcloss_reference(log_probs, targets, input_lengths, target_lengths).float()

        print("Result from Torch: %f" %result)
        print("Custom Function: %f" %expected)
        
        loss = result
        loss.backward()
        optimizer.step()
        
        train_loss += loss.detach().item()
        train_acc += 0
        #print("Loss: %f", %train_loss)
        
    #model.eval()
    print('training loss for epoch %d:' %epoch)
    
    epoch_loss = train_loss/len_data
    print(epoch_loss)

    

Result from Torch: 14.435223
Custom Function: 14.489079
Result from Torch: 13.657688
Custom Function: 12.693556
Result from Torch: 10.914380
Custom Function: 10.314322
Result from Torch: 8.301874
Custom Function: 7.928608
Result from Torch: 10.943694
Custom Function: 10.639611
Result from Torch: 11.159373
Custom Function: 10.790873
Result from Torch: 11.509882
Custom Function: 10.876806
Result from Torch: 8.418358
Custom Function: 8.708700
Result from Torch: 9.756591
Custom Function: 9.789805
Result from Torch: 9.903612
Custom Function: 8.985263
Result from Torch: 11.191595
Custom Function: 10.834159
Result from Torch: 8.492270
Custom Function: 8.334795
Result from Torch: 13.109535
Custom Function: 13.433983
Result from Torch: 13.169491
Custom Function: 12.642687
Result from Torch: 7.869191
Custom Function: 7.172386
Result from Torch: 12.471971
Custom Function: 12.545315
Result from Torch: 11.186040
Custom Function: 10.198828
Result from Torch: 8.007752
Custom Function: 7.330093
Result

Result from Torch: 8.927073
Custom Function: 8.509098
Result from Torch: 11.778308
Custom Function: 12.285869
Result from Torch: 10.046595
Custom Function: 9.427187
Result from Torch: 10.775304
Custom Function: 10.282290
Result from Torch: 11.056382
Custom Function: 11.057001
Result from Torch: 8.422318
Custom Function: 8.627394
Result from Torch: 9.081746
Custom Function: 8.688279
Result from Torch: 13.593740
Custom Function: 13.612267
Result from Torch: 11.128367
Custom Function: 10.997019
Result from Torch: 9.625879
Custom Function: 8.868359
Result from Torch: 12.776844
Custom Function: 11.995011
Result from Torch: 10.925145
Custom Function: 10.686485
Result from Torch: 12.102418
Custom Function: 12.039367
Result from Torch: 10.096049
Custom Function: 9.433962
Result from Torch: 12.437287
Custom Function: 11.514797
Result from Torch: 12.034166
Custom Function: 11.728661
Result from Torch: 11.539703
Custom Function: 11.345398
Result from Torch: 13.250466
Custom Function: 12.880612
Re

Result from Torch: 9.193078
Custom Function: 9.028284
Result from Torch: 11.885937
Custom Function: 11.403668
Result from Torch: 7.283710
Custom Function: 6.860809
Result from Torch: 9.880846
Custom Function: 9.610927
Result from Torch: 11.939405
Custom Function: 10.894753
Result from Torch: 11.176301
Custom Function: 11.597037
Result from Torch: 10.501842
Custom Function: 10.473885
Result from Torch: 11.137242
Custom Function: 11.708080
Result from Torch: 18.231047
Custom Function: 18.822865
Result from Torch: 11.601153
Custom Function: 11.913484
Result from Torch: 11.850716
Custom Function: 11.314749
Result from Torch: 11.142077
Custom Function: 11.073265
Result from Torch: 12.246453
Custom Function: 11.933688
Result from Torch: 14.344322
Custom Function: 13.218473
Result from Torch: 9.555344
Custom Function: 9.004310
Result from Torch: 13.681771
Custom Function: 12.796103
Result from Torch: 13.587099
Custom Function: 14.173907
Result from Torch: 11.716109
Custom Function: 11.828652


Result from Torch: 10.373951
Custom Function: 10.578471
Result from Torch: 9.819542
Custom Function: 9.283728
Result from Torch: 11.683854
Custom Function: 10.746329
Result from Torch: 8.955224
Custom Function: 8.945473
Result from Torch: 13.014841
Custom Function: 13.276950
Result from Torch: 9.508177
Custom Function: 9.547895
Result from Torch: 10.492925
Custom Function: 11.060071
Result from Torch: 16.694761
Custom Function: 18.083279
Result from Torch: 11.371801
Custom Function: 10.952500
Result from Torch: 11.752728
Custom Function: 11.815068
Result from Torch: 8.228661
Custom Function: 8.438867
Result from Torch: 8.985802
Custom Function: 9.694684
Result from Torch: 12.553587
Custom Function: 11.810248
Result from Torch: 10.849268
Custom Function: 10.430550
Result from Torch: 12.925918
Custom Function: 11.470375
Result from Torch: 9.248061
Custom Function: 8.950497
Result from Torch: 10.252829
Custom Function: 10.971466
Result from Torch: 11.706981
Custom Function: 10.944594
Resu

Result from Torch: 9.286280
Custom Function: 9.377286
Result from Torch: 11.502579
Custom Function: 11.363862
Result from Torch: 9.935589
Custom Function: 9.710695
Result from Torch: 16.477600
Custom Function: 16.584890
Result from Torch: 8.747313
Custom Function: 8.506614
Result from Torch: 11.359498
Custom Function: 12.200613
Result from Torch: 8.751569
Custom Function: 9.205216
Result from Torch: 9.572321
Custom Function: 9.650621
Result from Torch: 11.137778
Custom Function: 10.845326
Result from Torch: 10.431643
Custom Function: 10.332370
Result from Torch: 13.107761
Custom Function: 12.559564
Result from Torch: 10.209438
Custom Function: 10.235916
Result from Torch: 10.364717
Custom Function: 10.035668
Result from Torch: 20.702312
Custom Function: 21.938267
Result from Torch: 10.416973
Custom Function: 9.626801
Result from Torch: 9.441020
Custom Function: 8.900105
Result from Torch: 11.851645
Custom Function: 11.271560
Result from Torch: 11.856253
Custom Function: 11.489696
Resul

...

Result from Torch: 11.027039
Custom Function: 12.049346
Result from Torch: 16.259949
Custom Function: 17.990036
Result from Torch: 10.689173
Custom Function: 12.740635
Result from Torch: 17.242481
Custom Function: 17.876049
Result from Torch: 10.872878
Custom Function: 11.415853
Result from Torch: 12.866505
Custom Function: 15.441870
Result from Torch: 10.879822
Custom Function: 11.697009
Result from Torch: 14.341496
Custom Function: 17.713896
Result from Torch: 13.301713
Custom Function: 14.933051
Result from Torch: 11.659184
Custom Function: 12.941507
Result from Torch: 9.490280
Custom Function: 10.521568
Result from Torch: 12.911207
Custom Function: 14.901379
Result from Torch: 15.582918
Custom Function: 14.271162
Result from Torch: 10.795060
Custom Function: 10.469161
Result from Torch: 15.103848
Custom Function: 15.677865
Result from Torch: 13.532094
Custom Function: 16.017187
Result from Torch: 12.752475
Custom Function: 12.705681
Result from Torch: 13.940515
Custom Function: 15.

Result from Torch: 11.827987
Custom Function: 11.946080
Result from Torch: 9.953290
Custom Function: 10.730250
Result from Torch: 11.389548
Custom Function: 11.426403
Result from Torch: 10.166335
Custom Function: 10.072697
Result from Torch: 13.687984
Custom Function: 12.906885
Result from Torch: 13.464024
Custom Function: 15.720822
Result from Torch: 13.050140
Custom Function: 14.104525
Result from Torch: 10.700329
Custom Function: 10.908030
Result from Torch: 22.615191
Custom Function: 22.446268
Result from Torch: 13.309173
Custom Function: 13.815301
Result from Torch: 15.455316
Custom Function: 15.532478
Result from Torch: 13.096976
Custom Function: 2.314192
Result from Torch: 13.112232
Custom Function: 16.606327
Result from Torch: 12.223438
Custom Function: 13.758968
Result from Torch: 13.780263
Custom Function: 14.287049
Result from Torch: 14.686762
Custom Function: 13.753889
Result from Torch: 13.307905
Custom Function: 13.556183
Result from Torch: 12.287250
Custom Function: 12.2

Result from Torch: 11.577730
Custom Function: 11.391588
Result from Torch: 12.772508
Custom Function: 13.836673
Result from Torch: 9.640368
Custom Function: 10.149287
Result from Torch: 13.610735
Custom Function: 14.781510
Result from Torch: 12.914951
Custom Function: 14.051323
Result from Torch: 12.037644
Custom Function: 12.358202
Result from Torch: 12.925246
Custom Function: 12.518992
Result from Torch: 9.916340
Custom Function: 10.132346
Result from Torch: 14.501642
Custom Function: 14.588485
Result from Torch: 13.567740
Custom Function: 13.642422
Result from Torch: 10.923969
Custom Function: 12.981694
Result from Torch: 15.582853
Custom Function: 16.046537
Result from Torch: 10.750102
Custom Function: 11.849075
Result from Torch: 12.455454
Custom Function: 12.891133
Result from Torch: 10.248626
Custom Function: 10.675012
Result from Torch: 11.325171
Custom Function: 11.839859
Result from Torch: 11.504313
Custom Function: 12.507455
Result from Torch: 14.394998
Custom Function: 17.6

Result from Torch: 13.165546
Custom Function: 14.368800
Result from Torch: 17.512112
Custom Function: 16.045893
Result from Torch: 10.859131
Custom Function: 12.730780
Result from Torch: 14.520176
Custom Function: 15.205540
Result from Torch: 11.975669
Custom Function: 12.451341
Result from Torch: 14.061807
Custom Function: 14.434801
Result from Torch: 10.790192
Custom Function: 10.321641
Result from Torch: 16.999252
Custom Function: 19.036995
Result from Torch: 14.907929
Custom Function: 13.891223
Result from Torch: 13.469934
Custom Function: 12.821627
Result from Torch: 12.166736
Custom Function: 13.327001
Result from Torch: 11.514405
Custom Function: 11.903390
Result from Torch: 17.462778
Custom Function: 16.637846
Result from Torch: 12.064506
Custom Function: 13.946460
Result from Torch: 14.568393
Custom Function: 15.816399
Result from Torch: 9.821550
Custom Function: 10.312516
Result from Torch: 12.908540
Custom Function: 14.797617
Result from Torch: 17.301018
Custom Function: 18.

Result from Torch: 8.974615
Custom Function: 10.197768
Result from Torch: 10.994980
Custom Function: 10.622239
Result from Torch: 11.493391
Custom Function: 14.088706
Result from Torch: 16.066700
Custom Function: 18.191267
Result from Torch: 13.429399
Custom Function: 13.430941
Result from Torch: 13.677629
Custom Function: 12.505486
Result from Torch: 10.129653
Custom Function: 10.488923
Result from Torch: 11.306043
Custom Function: 12.650364
Result from Torch: 13.823387
Custom Function: 13.936142
Result from Torch: 13.743847
Custom Function: 14.268632
Result from Torch: 14.889609
Custom Function: 15.112616
Result from Torch: 15.322306
Custom Function: 19.213053
Result from Torch: 18.185469
Custom Function: 21.598150
Result from Torch: 10.991324
Custom Function: 11.015759
Result from Torch: 17.321449
Custom Function: 17.384626
Result from Torch: 9.435915
Custom Function: 9.917918
Result from Torch: 10.822876
Custom Function: 12.264928
Result from Torch: 11.001904
Custom Function: 12.02

In [118]:
# Save model
torch.save(model_cnn.state_dict(), "./cnn_model.pt")
torch.save(model_rnn.state_dict(), "./rnn_model.pt")

In [12]:
# Predict
test_model_cnn = cnn_model(BATCH_SIZE)
test_model_rnn = LoopRNN()

test_model_cnn.load_state_dict(torch.load('./cnn_model.pt'))
test_model_rnn.load_state_dict(torch.load('./rnn_model.pt'))

<All keys matched successfully>

In [20]:
# Test data

test_file_name = '/000051652-1_2_1'

path_string = corpus + test_file_name + test_file_name + '.png'

image = cv2.imread(path_string)

image = ctc_utils.resize(image, params['img_height'])

image = ctc_utils.normalize(image)

image = np.asarray(image).reshape(1, image.shape[0], image.shape[1] *3,1)



In [21]:
test_model_cnn.eval()
test_model_rnn.eval()

tensor_data = torch.from_numpy(image)

tensor_data_reshape = torch.permute(tensor_data,(0,3, 1, 2))

# forward, backward, optim
cnn_output = test_model_cnn(tensor_data_reshape)

output_to_loop = torch.reshape(cnn_output, (16, 1, cnn_output.shape[3], 64*14))

model_rnn.hidden = test_model_rnn.init_hidden(BATCH_SIZE)
loop_output = test_model_rnn(output_to_loop)

for t in range(0,len(loop_output)):
    loop_output[t] = torch.reshape(loop_output[t],(loop_output[t].size(1), loop_output[t].size(2)))

loop_tensor = torch.stack(loop_output)

# Reshape to correct shape for Medium article
loop_tensor = torch.permute(loop_tensor, (1,0,2))


# Convert to log softmax
loop_logits = log_softmax(loop_tensor)


RuntimeError: expected scalar type Double but found Float